In [1]:
import http.client
import json
import urllib.parse
import re
import pandas as pd
from shapely.geometry import Point, Polygon, MultiPolygon
import matplotlib.pyplot as plt
import numpy as np


In [2]:
# Function to parse the curl bash command into usable components
def clean_curl_request(curl_command):
    # Extract the URL
    url_pattern = r"curl '(.*?)'"
    match = re.search(url_pattern, curl_command)
    if match:
        url = match.group(1)
    else:
        raise ValueError("URL not found in the curl command")

    # Extract headers
    headers = {}
    header_pattern = r"-H '([^:]+): (.*?)'"
    for header_match in re.finditer(header_pattern, curl_command):
        header_name = header_match.group(1)
        header_value = header_match.group(2)
        headers[header_name] = header_value

    # Extract data
    data_pattern = r"--data-raw '(.*?)'"
    match = re.search(data_pattern, curl_command)
    data = {}
    if match:
        data_string = match.group(1)
        data_items = data_string.split("&")
        for item in data_items:
            key, value = item.split("=")
            data[key] = urllib.parse.unquote(value)

    # Parse URL for host and path
    parsed_url = urllib.parse.urlparse(url)
    host = parsed_url.netloc
    path = parsed_url.path + ("?" + parsed_url.query if parsed_url.query else "")

    return host, path, data, headers


In [3]:
# Function to send requests dynamically
def send_request(host, path, data, headers, lat, lng):
    # Update latitude and longitude in the data dictionary
    data["lat"] = str(lat)
    data["lng"] = str(lng)

    # Set up the connection
    connection = http.client.HTTPSConnection(host)

    # Send the POST request
    connection.request("POST", path, body=urllib.parse.urlencode(data), headers=headers)

    # Get the response
    response = connection.getresponse()
    response_data = response.read().decode()

    # Close the connection
    connection.close()

    # Check for a successful response and return the result
    if response.status == 200:
        return json.loads(response_data)
    else:
        return f"Error: {response.status}, {response_data}"

In [ ]:
def read_file():
    # Poligonun icindeki noktaları buraya ekleyecegiz.
    with open('istanbul-admin-level-6.geojson', 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    return data


In [ ]:
def get_properties(data,ilce):
    for i in data["features"]:
        print(i["properties"].get("name", "point"))
        # Polygonları alıyoruz sadece
        if i["geometry"].get("type", "unnamed") != "Point" and i["properties"].get("name", "point") == ilce:
            name = i["properties"].get("name", "point")
            geometry_type = i["geometry"].get("type", "point")
            coordinates = i["geometry"].get("coordinates", "point")
            # Polygon olanları direkt listeye atıyoruz.
            if geometry_type == "Polygon":
                polygons = [Polygon(coordinates[0])]
            # Multi-poly olanları listeler icinde listeye atıyoruz.
            elif geometry_type == "MultiPolygon":
                polygons = [Polygon(polygon[0]) for polygon in coordinates]
            else:
                raise ValueError(f"Unsupported geometry type: {geometry_type}")
    return name,geometry_type,polygons

In [ ]:
def extract_exterior_coordinates_create_lines(aralık=100,polygons):
    for polygon in polygons:
        # Extract exterior coordinates for plotting
        lat_spacing = aralik / 111320  # Approximation: 1 degree latitude ~ 111.32 km
        min_lat = min([min(polygon.exterior.coords.xy[1]) for polygon in polygons])
        max_lat = max([max(polygon.exterior.coords.xy[1]) for polygon in polygons])
        lat_lines = np.arange(min_lat, max_lat, lat_spacing)

        for lat in lat_lines:
            min_lon = min([min(polygon.exterior.coords.xy[0]) for polygon in polygons])
            max_lon = max([max(polygon.exterior.coords.xy[0]) for polygon in polygons])
            lon_spacing = aralik / (111320 * np.cos(np.radians(lat)))  # Adjust for longitude scaling
            lon_points = np.arange(min_lon, max_lon, lon_spacing)
    return lon_points,lat_lines

In [ ]:
def is_dot_inside_the_polygon(point_list):
    
    for lon in lon_points:
     point = Point(lon, lat)
     is_inside = any(polygon.contains(point) for polygon in polygons)
     if is_inside:
         point_list.append((lon, lat))
    
    return red_points

In [4]:
ilce = 'Adalar'
aralik = 100
# Listeyi loop'un icinde yaratıyoruz ki ilce ilce alabilelim.
red_points = []
# # Poligonun icindeki noktaları buraya ekleyecegiz.
# with open('istanbul-admin-level-6.geojson', 'r', encoding='utf-8') as file:
#     data = json.load(file)

#for i in data["features"]:
#    print(i["properties"].get("name", "point"))
#    # Polygonları alıyoruz sadece
#    if i["geometry"].get("type", "unnamed") != "Point" and i["properties"].get("name", "point") =! "Gebze":
#        name = i["properties"].get("name", "point")
#        geometry_type = i["geometry"].get("type", "point")
#        coordinates = i["geometry"].get("coordinates", "point")
#        # Polygon olanları direkt listeye atıyoruz.
#        if geometry_type == "Polygon":
#            polygons = [Polygon(coordinates[0])]
#        # Multi-poly olanları listeler icinde listeye atıyoruz.
#        elif geometry_type == "MultiPolygon":
#            polygons = [Polygon(polygon[0]) for polygon in coordinates]
#        else:
#            raise ValueError(f"Unsupported geometry type: {geometry_type}")

#        for polygon in polygons:
#            # Extract exterior coordinates for plotting
#            lat_spacing = aralik / 111320  # Approximation: 1 degree latitude ~ 111.32 km
#            min_lat = min([min(polygon.exterior.coords.xy[1]) for polygon in polygons])
#            max_lat = max([max(polygon.exterior.coords.xy[1]) for polygon in polygons])
#            lat_lines = np.arange(min_lat, max_lat, lat_spacing)
#
#            for lat in lat_lines:
#                min_lon = min([min(polygon.exterior.coords.xy[0]) for polygon in polygons])
#                max_lon = max([max(polygon.exterior.coords.xy[0]) for polygon in polygons])
#                lon_spacing = aralik / (111320 * np.cos(np.radians(lat)))  # Adjust for longitude scaling
#                lon_points = np.arange(min_lon, max_lon, lon_spacing)

                # Nokta poligonun icinde mi kontrol ediyoruz icindeyse kırmızı degilse yesil 
                for lon in lon_points:
                    point = Point(lon, lat)
                    is_inside = any(polygon.contains(point) for polygon in polygons)
                    if is_inside:
                        red_points.append((lon, lat))

        # DataFrame olustur ve verileri ekle
        columns = ["tesis_adi", "il_adi", "sokak_adi", "ilce_adi", "mahalle_adi", "lng", "lat", "id"]
        df = pd.DataFrame(columns=columns)
        for lng, lat in red_points:
            # Send API request
            result = send_request("host", "path", "data", "headers", lat, lng)
            # If the request is successful, process the results
            if isinstance(result, dict) and "features" in result:
                for feature in result["features"]:
                    # Extract relevant properties
                    properties = feature["properties"]
                    row = {
                        "tesis_adi": properties.get("tesis_adi", ""),
                        "il_adi": properties.get("il_adi", ""),
                        "sokak_adi": properties.get("sokak_adi", ""),
                        "ilce_adi": properties.get("ilce_adi", ""),
                        "mahalle_adi": properties.get("mahalle_adi", ""),
                        "lng": properties.get("lng", ""),
                        "lat": properties.get("lat", ""),
                        "id": properties.get("id", "")
                    }
                    # Append the row to the DataFrame
                    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

        # Drop duplicates after each iteration
        df.drop_duplicates(subset=["id"], inplace=True)

        # Save the final DataFrame to a CSV file
        df.to_csv(f"{name}_results.csv", index=False)


Adalar


TypeError: 'str' object does not support item assignment

In [ ]:
curl_command = """
curl 'https://www.turkiye.gov.tr/afet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama?harita=goster&submit' \
  -H 'Accept: application/json, text/javascript, */*; q=0.01' \
  -H 'Accept-Language: tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' \
  -H 'Cookie: ridbb=WyI1ZWZjYWJjN2E5YzNiY2M4ZDY4OWExMGIwN2IwMTcyMmNjZDc5NjUzZDBlOCJd; _uid=1721632846-0ba03683-f088-45be-ab36-f8dd67f8b32d; w3p=1943251136.20480.0000; language=tr_TR.UTF-8; uyari=b8e23373d48d32ab19013491277695691c051a6639e16343c50db3d9f71adf64; TURKIYESESSIONID=l4c0eutds26b2qr2ghqjf9mbt4; TS015d3f68=015c1cbb6d6bcb6358b3fc4d2387f6b25fa2d1245dfddce1e7b22bd8348a14a1e565feedf84c8944203eed55b58f837e9f56aacdc0; _lastptts=1735847586' \
  -H 'Origin: https://www.turkiye.gov.tr' \
  -H 'Referer: https://www.turkiye.gov.tr/afet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama?harita=goster' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-origin' \
  -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 OPR/115.0.0.0' \
  -H 'X-Requested-With: XMLHttpRequest' \
  -H 'sec-ch-ua: "Chromium";v="130", "Opera";v="115", "Not?A_Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Windows"' \
  --data-raw 'pn=%2Fafet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama&ajax=1&token=%7B6D7A6F-2ED4EB-F96A69-A23803-F7E63C-51CA4D-9AB3C3-BCD5F2%7D&islem=getAlanlarForNokta&lat=40.969583296311086&lng=29.095801512231148'
"""


# Parse the curl command
host, path, data, headers = clean_curl_request(curl_command)

In [ ]:


# Read coordinates from file and process them
with open('coordinates.txt', 'r') as file:
    for line in file:
        try:
            # Extract longitude and latitude
            lng, lat = map(float, line.strip().split(', '))

            # Send API request
            result = send_request(host, path, data, headers, lat, lng)

            # If the request is successful, process the results
            if isinstance(result, dict) and "features" in result:
                for feature in result["features"]:
                    # Extract relevant properties
                    properties = feature["properties"]
                    row = {
                        "tesis_adi": properties.get("tesis_adi", ""),
                        "il_adi": properties.get("il_adi", ""),
                        "sokak_adi": properties.get("sokak_adi", ""),
                        "ilce_adi": properties.get("ilce_adi", ""),
                        "mahalle_adi": properties.get("mahalle_adi", ""),
                        "lng": properties.get("lng", ""),
                        "lat": properties.get("lat", ""),
                        "id": properties.get("id", "")
                    }
                    # Append the row to the DataFrame
                    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

            # Drop duplicates after each iteration
            df.drop_duplicates(subset=["id"], inplace=True)

        except Exception as e:
            print(f"Error processing line '{line.strip()}': {e}")

# Save the final DataFrame to a CSV file
df.to_csv("api_results.csv", index=False)
